# Fine-tuning a model with the Trainer API

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

datasets : pour charger des jeux de données prêts à l'emploi
evaluate : pour mesurer les performances du modèle
transformers[sentencepiece] : la bibliothèque principale qui contient BERT et tout le reste.
[sentencepiece] installe un outil de tokenisation supplémentaire.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


 on charge le dataset MRPC (Microsoft Research Paraphrase Corpus). C'est un jeu de données de paires de phrases, et pour chaque paire, on sait si elles veulent dire la même chose (label 1) ou non (label 0). Il contient 3 parties : train, validation, test.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [6]:
trainer.train()

Step,Training Loss
500,0.545923
1000,0.296452


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1377, training_loss=0.3415084318051584, metrics={'train_runtime': 246.0531, 'train_samples_per_second': 44.722, 'train_steps_per_second': 5.596, 'total_flos': 405114969714960.0, 'train_loss': 0.3415084318051584, 'epoch': 3.0})

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [8]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [9]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8627450980392157, 'f1': 0.9041095890410958}

In [10]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
trainer.train()

fine tunnig

Transformers
C'est la bibliothèque la plus importante du notebook. Elle contient les grands modèles de langage comme BERT, GPT, etc., et tous les outils pour les utiliser, les modifier, les entraîner. Aussi créée par Hugging Face.

[sentencepiece]
Les crochets après transformers veulent dire "installe aussi ce module optionnel en plus". sentencepiece est un outil de découpe de texte en morceaux (tokens) utilisé par certains modèles comme T5 ou mBERT

Load_dataset
C'est une fonction (un outil qu'on appelle avec des parenthèses). Elle se connecte à internet, va sur le site de Hugging Face, et télécharge un jeu de données. Sans cette fonction, tu devrais trouver le dataset toi-même, le télécharger, comprendre son format (CSV? JSON? autre?), écrire du code pour le lire... load_dataset fait tout ça automatiquement.

"glue"
GLUE = General Language Understanding Evaluation.
GLUE contient plusieurs tâches différentes (détecter des contradictions, répondre à des questions, etc.).
parmis les taches de glue one trouve :

"mrpc"
MRPC = Microsoft Research Paraphrase Corpus. C'est une des tâches dans GLUE. Le dataset contient des paires de phrases extraites de journaux en ligne. Pour chaque paire, un humain a jugé si les deux phrases disent la même chose (paraphrase) ou non.

checkpoint
En deep learning, un checkpoint c'est une sauvegarde d'un modèle à un moment donné de son entraînement. Ici, "bert-base-uncased" est le nom du checkpoint sur Hugging Face Hub (le site où sont stockés des milliers de modèles pré-entraînés).chckpoint ici c'est le modele LLM utilisé
Entraîner BERT depuis zéro a pris à Google des semaines sur des centaines d'ordinateurs puissants et coûté des millions de dollars. À la fin, ils ont sauvegardé l'état complet du modèle (tous ses millions de paramètres, les réglages de ses boutons). Cette sauvegarde, c'est le checkpoint.
Ce  fichier de sauvegarde a été mis en ligne sur Hugging Face Hub (un site comme GitHub mais pour les modèles IA) sous le nom "bert-base-uncased".
**********************************************************************

# Utilisation 1 : télécharger le tokenizer de BERT
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#                                          ↑
#                         "va chercher le tokenizer de bert-base-uncased"

# Utilisation 2 : télécharger le modèle BERT lui-même
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
#                                           ↑
#                         "va chercher le modèle bert-base-uncased"
************************************************************************

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
Qu'est-ce qu'un token ?
Un modèle IA ne lit pas du texte. Il ne comprend que des nombres. Il faut donc transformer chaque phrase en une liste de nombres. Un token est un morceau de texte (souvent un mot entier, parfois une partie de mot) auquel on associe un numéro.

AutoTokenizer
C'est une classe "intelligente" de Hugging Face. Le préfixe Auto veut dire : "devine automatiquement quel tokenizer utiliser en fonction du modèle choisi". Si demain tu changes checkpoint pour "gpt2", AutoTokenizer chargera le bon tokenizer pour GPT-2 sans que tu changes quoi que ce soit d'autre.
.from_pretrained(checkpoint)
Cette méthode (fonction attachée à un objet) va sur internet, télécharge la configuration du tokenizer de BERT et le crée prêt à l'emploi. from_pretrained = "crée cet objet depuis une version déjà entraînée/configurée".



def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
tokenizer(une_seule_chose)        # → sépare seulement

tokenizer(chose1, chose2)         # → sépare ET concatène

example
C'est le paramètre de la fonction. Quand on appellera tokenize_function, on lui passera un exemple du dataset (une ligne). Cet exemple ressemble à un dictionnaire Python : {"sentence1": "...", "sentence2": "...", "label": 1}.
example["sentence1"] et example["sentence2"]
On accède aux deux phrases de la paire. ["sentence1"] = "donne-moi la valeur associée à la clé sentence1 dans ce dictionnaire".
tokenizer(sentence1, sentence2, ...)
Quand on donne deux phrases au tokenizer, il les fusionne intelligemment en ajoutant des tokens spéciaux pour que BERT sache où commence et où finit chaque phrase. Le résultat ressemble à : [CLS] phrase1 [SEP] phrase2 [SEP] où [CLS] et [SEP] sont des tokens spéciaux de BERT.
truncation=True
BERT a une limite : il ne peut pas traiter plus de 512 tokens à la fois. Si une paire de phrases est trop longue, truncation=True dit "coupe-la automatiquement" plutôt que de planter avec une erreur.

.map()
C'est une méthode qui applique une fonction à chaque élément d'un dataset. Comme si tu avais 3668 paires de phrases et que tu appliquais tokenize_function sur chacune, une par une. .map() le fait automatiquement pour toutes.


batched=True
Au lieu de traiter les exemples un par un, on les traite par groupes (batch = lot). C'est beaucoup plus rapide parce que le tokenizer est optimisé pour traiter des lots.
tokenized_datasets
C'est le dataset original, mais maintenant chaque exemple contient en plus les tokens (les nombres) en plus des phrases texte.



